In [1]:
!pip install conllu

In [2]:
!pip install --force-reinstall --no-dependencies "scikit-learn==0.24.2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 41.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [3]:
!pip install sklearn_crfsuite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.5 MB/s eta 0:00:00


In [4]:
import conllu
from conllu import parse, parse_incr
from io import open

from collections import Counter
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [5]:
#  корпус SynTagRus
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu

--2024-03-20 23:21:40--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736599 (39M) [text/plain]
Saving to: ‘ru_syntagrus-ud-train-a.conllu’

ru_syntagrus-ud-tra 100%[===================>]  38.85M   119MB/s    in 0.3s    

2024-03-20 23:21:43 (119 MB/s) - ‘ru_syntagrus-ud-train-a.conllu’ saved [40736599/40736599]

--2024-03-20 23:21:43--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-test.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.19

In [6]:
def read_conllu_file(data_file_path):
    sentences = []
    current_sentence = []

    with open(data_file_path, "r", encoding="utf8") as file:
        for line in file:
            stripped_line = line.strip()
            if stripped_line:
                current_sentence.append(stripped_line)
            elif current_sentence:
                sentences.append(current_sentence)
                current_sentence = []

    # добавляем последнее предложение, если оно существует
    if current_sentence:
        sentences.append(current_sentence)

    return sentences

# путь к обучающему набору данных
training_data_path = "ru_syntagrus-ud-train-a.conllu"
sentences = read_conllu_file(training_data_path)
print("\n".join(sentences[5]))

# sent_id = 2003Anketa.xml_6
# text = У двери стоял стол секретарши, на столе - пишущая машинка с широкой кареткой.
1	У	у	ADP	_	_	2	case	2:case	_
2	двери	дверь	NOUN	_	Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing	3	obl	3:obl:у:gen	_
3	стоял	стоять	VERB	_	Aspect=Imp|Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	0:root	_
4	стол	стол	NOUN	_	Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing	3	nsubj	3:nsubj	_
5	секретарши	секретарша	NOUN	_	Animacy=Anim|Case=Gen|Gender=Fem|Number=Sing	4	nmod	4:nmod:gen	SpaceAfter=No
6	,	,	PUNCT	_	_	8	punct	8:punct	_
7	на	на	ADP	_	_	8	case	8:case	_
8	столе	стол	NOUN	_	Animacy=Inan|Case=Loc|Gender=Masc|Number=Sing	3	conj	0:root|3:conj	_
9	-	-	PUNCT	_	_	11	punct	11:punct	_
10	пишущая	писать	VERB	_	Aspect=Imp|Case=Nom|Gender=Fem|Number=Sing|Tense=Pres|VerbForm=Part|Voice=Act	11	amod	11:amod	_
11	машинка	машинка	NOUN	_	Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing	8	nsubj	8:nsubj	_
12	с	с	ADP	_	_	14	case	14:case	_
13	широкой	широкий	ADJ	_	Ca

In [7]:
# превращаем в список
with open("ru_syntagrus-ud-train-a.conllu", "r", encoding="utf-8") as f:
    for tokenlist in parse_incr(f):
      print(list((tokenlist)))
      break

[{'id': 1, 'form': 'Анкета', 'lemma': 'анкета', 'upos': 'NOUN', 'xpos': None, 'feats': {'Animacy': 'Inan', 'Case': 'Nom', 'Gender': 'Fem', 'Number': 'Sing'}, 'head': 0, 'deprel': 'root', 'deps': [('root', 0)], 'misc': {'SpaceAfter': 'No'}}, {'id': 2, 'form': '.', 'lemma': '.', 'upos': 'PUNCT', 'xpos': None, 'feats': None, 'head': 1, 'deprel': 'punct', 'deps': [('punct', 1)], 'misc': None}]


In [8]:
# indexing
with open("ru_syntagrus-ud-train-a.conllu", "r", encoding="utf-8") as f:
    for tokenlist in parse_incr(f):
      print(tokenlist[0], tokenlist[0]['upos'])
      break

Анкета NOUN


In [9]:
def word2features(sent, index):
    word = sent[index]['form']

    # основные признаки слова
    features = {
        'bias': 1.0,
        'word.lower': word.lower(),
        'word.suffix3': word[-3:],
        'word.suffix2': word[-2:],
        'word.isupper': word.isupper(),
        'word.istitle': word.istitle(),
        'word.isdigit': word.isdigit(),
    }

    # признаки предыдущего слова
    if index > 0:
        prev_word = sent[index-1]['form']
        features.update({
            'prev_word.lower': prev_word.lower(),
            'prev_word.istitle': prev_word.istitle(),
            'prev_word.isupper': prev_word.isupper(),
        })
    else:
        # для начала предложения
        features['start_of_sentence'] = True

    # признаки следующего слова
    if index < len(sent) - 1:
        next_word = sent[index+1]['form']
        features.update({
            'next_word.lower': next_word.lower(),
            'next_word.istitle': next_word.istitle(),
            'next_word.isupper': next_word.isupper(),
        })
    else:
        # для конца предложения
        features['end_of_sentence'] = True

    return features

In [13]:
def sent2features(sent): # обработка предложений
    return [word2features(sent, index) for index in range(len(sent))]

def sent2labels(sent):  # достаем pos-тэг
    return [word['upos'] for word in sent]

In [14]:
X_train, y_train, X_test, y_test = [], [], [], []

In [15]:
with open("ru_syntagrus-ud-train-a.conllu", "r", encoding="utf-8") as f:
    for tokenlist in parse_incr(f):
      X_train.append(sent2features(tokenlist))
      y_train.append(sent2labels(tokenlist))

In [16]:
with open("ru_syntagrus-ud-test.conllu", "r", encoding="utf-8") as f:
    for tokenlist in parse_incr(f):
      X_test.append(sent2features(tokenlist))
      y_test.append(sent2labels(tokenlist))

In [17]:
print(X_train[0])
print(y_train[0])

[{'bias': 1.0, 'word.lower': 'анкета', 'word.suffix3': 'ета', 'word.suffix2': 'та', 'word.isupper': False, 'word.istitle': True, 'word.isdigit': False, 'start_of_sentence': True, 'next_word.lower': '.', 'next_word.istitle': False, 'next_word.isupper': False}, {'bias': 1.0, 'word.lower': '.', 'word.suffix3': '.', 'word.suffix2': '.', 'word.isupper': False, 'word.istitle': False, 'word.isdigit': False, 'prev_word.lower': 'анкета', 'prev_word.istitle': True, 'prev_word.isupper': False, 'end_of_sentence': True}]
['NOUN', 'PUNCT']


In [18]:
crf = sklearn_crfsuite.CRF(algorithm='lbfgs', c1=0.1, c2=0.001, max_iterations=50, all_possible_transitions=True)

# обучение модели
crf.fit(X_train, y_train)

# предсказания на тестовом наборе данных
predictions = crf.predict(X_test)

In [19]:
y_preds = crf.predict(X_test)

In [20]:
labels = list(crf.classes_)

In [21]:
metrics.flat_f1_score(y_test, y_preds,
                      average='weighted', labels=labels)

0.9658665358330675

In [22]:
from sklearn_crfsuite.utils import flatten
from sklearn import metrics as skmetrics

y_test_flat = flatten(y_test)
y_pred_flat = flatten(y_preds)

print(skmetrics.classification_report(y_test_flat, y_pred_flat, labels = labels))

              precision    recall  f1-score   support

        NOUN       0.98      0.98      0.98     36568
       PUNCT       1.00      1.00      1.00     29463
         ADJ       0.91      0.95      0.93     14471
       PROPN       0.95      0.95      0.95      5883
         AUX       0.93      0.93      0.93      1518
        VERB       0.97      0.95      0.96     18146
         ADP       1.00      1.00      1.00     15062
         ADV       0.93      0.94      0.94      8085
       CCONJ       0.95      0.98      0.96      5736
        PART       0.95      0.88      0.91      4921
        PRON       0.96      0.95      0.95      8015
         DET       0.94      0.88      0.91      4094
       SCONJ       0.90      0.95      0.92      2992
         NUM       0.95      0.94      0.95      2528
           _       1.00      1.00      1.00       271
        INTJ       1.00      0.30      0.47        23
           X       0.95      0.38      0.54        48
         SYM       1.00    